
|Version|Date|Description|
|--|--|--|
|0.1|2024-11-29|Initial|
|0.2|2024-12-03|Updated the model parameter by increasing the no.of stack LSTM layer from 1 to 3|

Training Model

Utilize the historical data from King's Park to train and evaluate the model. The sequence length is set to 7 days, meaning the past 7 days of data are used to predict the HKHI index for the next day.

Fine Tunning Model

Partition the full dataset by region. For each region, use only the most recent 7 days of data to fine-tune the model and predict the HKHI index for the following day.

In [1]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [8]:
path = "/content/drive/MyDrive/CUHK/STAT 6207 Deep Learning/Project/"

In [9]:
df = pd.read_csv(path+"DAILY_FINAL.csv")

<ipython-input-9-cfe0b553b345>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+"DAILY_FINAL.csv")


In [10]:
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']]).dt.date

In [11]:
df = df.drop(['Year', 'Month', 'Day'], axis=1)


In [12]:
kings_park_data = df[(df['WeatherStationName_en'] == "King's Park") & (df['Value_Heat_Index'].notna())]

In [13]:
kings_park_data = kings_park_data.sort_values(by = 'Date').dropna()

In [14]:
kings_park_data.count()

,0
WeatherStationName_en,3774
Value_Humidity,3774
Value_Temperature,3774
Value_Wind_Speed,3774
Value_Wind_Direction,3774
Value_Heat_Index,3774
Date,3774


In [15]:
features = ['Value_Humidity','Value_Temperature', 'Value_Wind_Speed', 'Value_Wind_Direction']
target = 'Value_Heat_Index'

In [16]:
X = kings_park_data[features].values
y = kings_park_data[target].values

In [21]:
kings_park_data.isna().sum()

,0
WeatherStationName_en,0
Value_Humidity,0
Value_Temperature,0
Value_Wind_Speed,0
Value_Wind_Direction,0
Value_Heat_Index,0
Date,0


In [24]:
kings_park_data.tail()

,WeatherStationName_en,Value_Humidity,Value_Temperature,Value_Wind_Speed,Value_Wind_Direction,Value_Heat_Index,Date
143146,King's Park,79.0,29.1,9.2,280.0,28.0,2024-09-26
143158,King's Park,77.0,29.7,9.6,270.0,28.1,2024-09-27
143171,King's Park,82.0,28.7,10.2,110.0,27.5,2024-09-28
143184,King's Park,76.0,28.9,7.1,280.0,27.6,2024-09-29
143197,King's Park,69.0,30.3,9.0,270.0,28.1,2024-09-30


In [23]:
kings_park_data.query("Value_Heat_Index.isna()")


,WeatherStationName_en,Value_Humidity,Value_Temperature,Value_Wind_Speed,Value_Wind_Direction,Value_Heat_Index,Date


In [18]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [22]:
# Check for non-numeric values in the features (X)
print(kings_park_data[features].applymap(type).value_counts())

# Check for non-numeric values in the target (y)
print(kings_park_data[target].apply(type).value_counts())

Value_Humidity   Value_Temperature  Value_Wind_Speed  Value_Wind_Direction
<class 'float'>  <class 'float'>    <class 'float'>   <class 'float'>         3774
Name: count, dtype: int64
Value_Heat_Index
<class 'str'>      2847
<class 'float'>     927
Name: count, dtype: int64


<ipython-input-22-c0bc08493223>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(kings_park_data[features].applymap(type).value_counts())


In [19]:
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

ValueError: could not convert string to float: '***'

In [ ]:
# Create sequences for LSTM
def create_sequences(X, y, seq_length=7):
    X_seq, y_seq = [], []
    for i in range(len(X) - seq_length):
        X_seq.append(X[i:i+seq_length])
        y_seq.append(y[i+seq_length])
    return np.array(X_seq), np.array(y_seq)

In [ ]:
seq_length = 7
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

In [ ]:
# Train-test split
train_size = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]


In [ ]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [ ]:
# Define the Improved LSTM Model
class ImprovedLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout=0.2, bidirectional=True):
        super(ImprovedLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # LSTM layer with multiple layers, dropout, and bidirectionality
        self.lstm = nn.LSTM(
            input_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=bidirectional
        )

        # Fully connected layer (adjust input size for bidirectional LSTM)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden and cell states for all layers (including forward and backward states if bidirectional)
        h0 = torch.zeros(
            self.num_layers * 2 if self.bidirectional else self.num_layers,
            x.size(0),
            self.hidden_dim
        ).to(device)

        c0 = torch.zeros(
            self.num_layers * 2 if self.bidirectional else self.num_layers,
            x.size(0),
            self.hidden_dim
        ).to(device)

        # LSTM forward pass
        out, _ = self.lstm(x, (h0, c0))

        # Fully connected layer on the last time step
        out = self.fc(out[:, -1, :])
        return out

In [ ]:
# Model parameters
input_dim = len(features)  # Number of features
hidden_dim = 128           # Increased number of hidden units for more capacity
output_dim = 1             # Predicting a single value (HKHI index)
num_layers = 3             # Increased depth with 3 LSTM layers
dropout = 0.3              # Dropout probability to prevent overfitting
bidirectional = True       # Use bidirectional LSTM

# Instantiate the improved model
model = ImprovedLSTMModel(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    num_layers=num_layers,
    dropout=dropout,
    bidirectional=bidirectional
).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# Train the model
num_epochs = 50
train_losses = []

In [ ]:
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


In [ ]:
# Plot training loss
plt.plot(train_losses, label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Curve")
plt.legend()
plt.show()

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f"Test Loss: {test_loss.item():.4f}")


In [ ]:
# Inverse transform the test labels
y_test_original = scaler_y.inverse_transform(y_test_tensor.cpu().numpy())

In [ ]:
# Inverse transform the predicted values
y_pred_original = scaler_y.inverse_transform(y_pred.cpu().numpy())

In [ ]:
# Extract datetime values for the test data
# Assuming `kings_park_data` contains the original dataset with 'Year', 'Month', and 'Day'.
test_dates = kings_park_data.iloc[-len(y_test_original):]['Date']


# Plot actual vs predicted with datetime on the x-axis
plt.figure(figsize=(12, 6))
plt.plot(test_dates, y_test_original, label="Actual", color="blue")
plt.plot(test_dates, y_pred_original, label="Predicted", color="orange", alpha=0.7)

plt.xlabel("Date")
plt.ylabel("HKHI Index")
plt.title("Actual vs Predicted HKHI Index")
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

Predict Other regions

In [ ]:
other_regions_data = df[df['WeatherStationName_en'] != "King's Park"]

In [ ]:
other_regions_data = other_regions_data.sort_values(by='Date', ascending=False)


In [ ]:
latest_7_days_by_region = (
    other_regions_data.groupby('WeatherStationName_en')
    .head(7)  # Get the top 7 rows for each group (region)
    .sort_values(by=['WeatherStationName_en', 'Date'])  # Optional: Re-sort the result by region and date
)

In [ ]:
latest_7_days_by_region

In [ ]:
# Initialize an empty list to store predictions
predictions = []

# Group by region
for region, group in latest_7_days_by_region.groupby('WeatherStationName_en'):
    # Select features and scale
    X_latest = group[features].values
    X_latest_scaled = scaler_X.transform(X_latest)

    # Prepare the input sequence
    X_latest_sequence = X_latest_scaled.reshape(1, seq_length, len(features))  # Shape: (1, 7, num_features)
    X_latest_tensor = torch.tensor(X_latest_sequence, dtype=torch.float32).to(device)

    # Predict the next day's HKHI index
    model.eval()
    with torch.no_grad():
        next_day_prediction = model(X_latest_tensor)

    # Inverse transform the prediction
    next_day_prediction_original_scale = scaler_y.inverse_transform(next_day_prediction.cpu().numpy())

    # Get the latest date in this region's data
    latest_date = group['Date'].max()

    # Calculate the prediction date (next day)
    predicted_date = pd.to_datetime(latest_date) + pd.Timedelta(days=1)

    # Save the prediction, region, and predicted date
    predictions.append({
        'Region': region,
        'Predicted_Date': predicted_date.strftime('%Y-%m-%d'),
        'Predicted_HKHI_Index': next_day_prediction_original_scale[0][0]
    })

In [ ]:
# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions)

# Display predictions
predictions_df.head(100)

In [ ]:
# Save the trained model
torch.save(model.state_dict(), "lstm_model.pth")

# Load the model
model.load_state_dict(torch.load("lstm_model.pth"))
model.eval()

In [ ]:
predictions_df.to_csv('FACT_FINAL.csv', index=False)